In [1]:
from xml.etree import ElementTree
from xml.dom import minidom
from xml.etree.ElementTree import Element, SubElement, Comment, indent

def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ElementTree.tostring(elem, encoding="ISO-8859-1")
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="\t")

In [24]:
import numpy as np
import os

valve_start = 1

hyb_start = 50

reg_start = 1

num_rounds = 14

data_type = "U"#'C'

valve_ids = np.arange(valve_start, valve_start + num_rounds)
hyb_ids = np.arange(hyb_start, hyb_start + num_rounds)
reg_names = [f'{data_type}{_i}' for _i in np.arange(reg_start, reg_start + num_rounds)]

In [25]:
source_folder = r'D:\Pu\20220215-P_brain_CTP11-1000_CTP12_from0208'
#target_drive = r'\\KOLMOGOROV\Chromatin_NAS_4'
target_drive = r'\\10.245.74.158\Chromatin_NAS_0'

# imaging protocol
imaging_protocol = r'Zscan_750_647_561_s50_n250_10Hz'
# reference imaging protocol
add_ref_hyb = False

ref_imaging_protocol = r'scan_750_647_561_405_s50_n250_10Hz'
ref_hyb = 0
# bleach protocol
bleach = True
bleach_protocol = r'Bleach_750_647_s5'

In [26]:
cmd_seq = Element('command_sequence')

if add_ref_hyb:
    # add hyb 0
    # comments
    comment = Comment(f"Hyb 0")
    cmd_seq.append(comment)
    # flow imaging buffer
    imaging = SubElement(cmd_seq, 'valve_protocol')
    imaging.text = f"Flow Imaging Buffer"

    # change directory
    change_dir = SubElement(cmd_seq, 'change_directory')
    change_dir.text = os.path.join(source_folder, f"H0C1")
    # wakeup
    wakeup = SubElement(cmd_seq, 'wakeup')
    wakeup.text = "5000"

    # imaging loop
    _im_p = ref_imaging_protocol

    loop = SubElement(cmd_seq, 'loop', name='Position Loop Zscan', increment="name")
    loop_item = SubElement(loop, 'item', name=_im_p)
    loop_item.text = " "
    # delay time
    delay = SubElement(cmd_seq, 'delay')
    delay.text = "2000"
    # copy folder
    copy_dir = SubElement(cmd_seq, 'copy_directory')
    source_dir = SubElement(copy_dir, 'source_path')
    source_dir.text = change_dir.text 
    target_dir = SubElement(copy_dir, 'target_path')
    target_dir.text = os.path.join(target_drive, 
                                   os.path.basename(os.path.dirname(source_dir.text)), 
                                   os.path.basename(source_dir.text))
    del_source = SubElement(copy_dir, 'delete_source')
    del_source.text = "True"


for _i, (_vid, _hid, _rname) in enumerate(zip(valve_ids, hyb_ids, reg_names)):
    # select protocol
    _im_p = imaging_protocol
    # TCEP
    tcep = SubElement(cmd_seq, 'valve_protocol')
    tcep.text = "Flow TCEP"
    # wash tcep
    tcep_wash = SubElement(cmd_seq, 'valve_protocol')
    tcep_wash.text = "Flow Wash Buffer"
        
    # comments
    comment = Comment(f"Hyb {_hid} with {_vid} for {_rname}")
    cmd_seq.append(comment)
    # flow adaptor
    adt = SubElement(cmd_seq, 'valve_protocol')
    adt.text = f"Hybridize {_vid}"
    if bleach:
        # delay time
        delay = SubElement(cmd_seq, 'delay')
        #delay.text = "30000"
        delay.text = "35000"
        # change directory
        bleach_change_dir = SubElement(cmd_seq, 'change_directory')
        bleach_change_dir.text = os.path.join(source_folder, f"Bleach")
        # wakeup
        bleach_wakeup = SubElement(cmd_seq, 'wakeup')
        bleach_wakeup.text = "5000"
        # imaging loop
        bleach_loop = SubElement(cmd_seq, 'loop', name='Position Loop Zscan', increment="name")
        bleach_loop_item = SubElement(bleach_loop, 'item', name=bleach_protocol)
        bleach_loop_item.text = " "
        # delay time
        delay = SubElement(cmd_seq, 'delay')
        delay.text = "3000"
    else:
        # delay time
        adt_incubation = SubElement(cmd_seq, 'valve_protocol')
        adt_incubation.text = f"Incubate 10min"
    # wash
    wash = SubElement(cmd_seq, 'valve_protocol')
    wash.text = "Flow Wash Buffer"
    
    # readouts
    readouts = SubElement(cmd_seq, 'valve_protocol')
    readouts.text = "Flow RNA common readouts"
    # incubate readouts
    readout_incubation = SubElement(cmd_seq, 'valve_protocol')
    readout_incubation.text = f"Incubate 10min"
    # wash readouts
    readout_wash = SubElement(cmd_seq, 'valve_protocol')
    readout_wash.text = f"Flow Wash Buffer"
    # flow imaging buffer
    imaging = SubElement(cmd_seq, 'valve_protocol')
    imaging.text = f"Flow Imaging Buffer"
    
    # change directory
    change_dir = SubElement(cmd_seq, 'change_directory')
    change_dir.text = os.path.join(source_folder, f"H{_hid}{_rname.upper()}")
    # wakeup
    wakeup = SubElement(cmd_seq, 'wakeup')
    wakeup.text = "5000"
    
    # imaging loop
    loop = SubElement(cmd_seq, 'loop', name='Position Loop Zscan', increment="name")
    loop_item = SubElement(loop, 'item', name=_im_p)
    loop_item.text = " "
    # delay time
    delay = SubElement(cmd_seq, 'delay')
    delay.text = "2000"
    # copy folder
    copy_dir = SubElement(cmd_seq, 'copy_directory')
    source_dir = SubElement(copy_dir, 'source_path')
    source_dir.text = change_dir.text#cmd_seq.findall('change_directory')[-1].text
    target_dir = SubElement(copy_dir, 'target_path')
    target_dir.text = os.path.join(target_drive, 
                                   os.path.basename(os.path.dirname(source_dir.text)), 
                                   os.path.basename(source_dir.text))
    del_source = SubElement(copy_dir, 'delete_source')
    del_source.text = "True"
    # empty line
    indent(target_dir)

final_str = prettify(cmd_seq)
    
print( final_str )

<?xml version="1.0" ?>
<command_sequence>
	<valve_protocol>Flow TCEP</valve_protocol>
	<valve_protocol>Flow Wash Buffer</valve_protocol>
	<!--Hyb 50 with 1 for U1-->
	<valve_protocol>Hybridize 1</valve_protocol>
	<delay>35000</delay>
	<change_directory>D:\Pu\20220215-P_brain_CTP11-1000_CTP12_from0208\Bleach</change_directory>
	<wakeup>5000</wakeup>
	<loop name="Position Loop Zscan" increment="name">
		<item name="Bleach_750_647_s5"> </item>
	</loop>
	<delay>3000</delay>
	<valve_protocol>Flow Wash Buffer</valve_protocol>
	<valve_protocol>Flow RNA common readouts</valve_protocol>
	<valve_protocol>Incubate 10min</valve_protocol>
	<valve_protocol>Flow Wash Buffer</valve_protocol>
	<valve_protocol>Flow Imaging Buffer</valve_protocol>
	<change_directory>D:\Pu\20220215-P_brain_CTP11-1000_CTP12_from0208\H50U1</change_directory>
	<wakeup>5000</wakeup>
	<loop name="Position Loop Zscan" increment="name">
		<item name="Zscan_750_647_561_s50_n250_10Hz"> </item>
	</loop>
	<delay>2000</delay>
	<copy_

# save this xml

In [27]:
save_filename = os.path.join(source_folder, f"generated_dave_H{hyb_start}-{hyb_start+num_rounds-1}.txt")
with open(save_filename, 'w') as _output_handle:
    print(save_filename)
    _output_handle.write(final_str)
    

D:\Pu\20220215-P_brain_CTP11-1000_CTP12_from0208\generated_dave_H50-63.txt
